In [1]:
from transformers import BertTokenizer, BertForMaskedLM, AutoTokenizer
import torch
import pandas as pd

In [2]:
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9b1f43542707628fc961b5fd804009acefe9a67f08505e9d4b637c558b250dd1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
df = pd.read_table('/kaggle/input/paranmt/filtered.tsv')
df.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import numpy as np
from transformers import RobertaForSequenceClassification, RobertaTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

def predict_toxicity(texts, device='cpu', clf_name = 's-nlp/roberta_toxicity_classifier_v1'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device)
    clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out


def mask_toxic(sentence, threshold=0.3):
    words = sentence.split()
    probabilities = predict_toxicity(words)
    text_prep = []
    toxic_indexes = []
    for _word, _prob in zip(words, probabilities):
        if _prob > threshold:
            text_prep.append("[MASK]")
        else:
            text_prep.append(_word)
    text_prep = " ".join(text_prep)
    tokenized = tokenizer(text_prep, return_tensors="pt")
    return tokenized

def get_mask_index(inputs):
    mask_token_indexes = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    return mask_token_indexes

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def infer(sentence):
    inputs = mask_toxic(sentence)
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_indexes = get_mask_index(inputs)
    for mask_token_index in mask_indexes:
        predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
        inputs.input_ids[0][mask_token_index] = predicted_token_id
    decoded_sentence = tokenizer.decode(inputs.input_ids[0][1:-1])
    return decoded_sentence

In [7]:
inputs = infer("I hate idiots and you")
print(inputs)

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


i hate you and you


In [8]:
from datasets import load_metric
import evaluate

sacrebleu_metric = load_metric("sacrebleu")
rouge_metric = evaluate.load('rouge')
ter_metric = evaluate.load("ter")

In [9]:
from tqdm import trange
from transformers import logging

logging.set_verbosity_error()

toxic_sentences = df.reference[:200].tolist()
toxic_sentences_list = [[t] for t in toxic_sentences]
detoxified_text = []
result = {
    "bleu": 0,
    "rouge1": 0,
    "rouge2": 0,
    "TER": 0
}
n = len(toxic_sentences)
for i in trange(len(toxic_sentences)):
    detoxified_sentence = infer(toxic_sentences[i])
    #words = toxic_sentences[i].split()
    #toxic_scores = predict_toxicity(words)
    #detoxified_sentence = " ".join([sentence for sentence, score in zip(words, toxic_scores) if score < threshold])
    detoxified_text.append(detoxified_sentence)

result["bleu"] = sacrebleu_metric.compute(predictions=detoxified_text, references=toxic_sentences_list)["score"]
rouge_score = rouge_metric.compute(predictions=detoxified_text, references=toxic_sentences_list)
result["rouge1"] = rouge_score["rouge1"]
result["rouge2"] = rouge_score["rouge2"]
result["TER"] = ter_metric.compute(predictions=detoxified_text, references=toxic_sentences_list)["score"]

100%|██████████| 200/200 [05:28<00:00,  1.64s/it]


In [10]:
print(result)

{'bleu': 67.59722761285171, 'rouge1': 0.9580604262426111, 'rouge2': 0.9075920886922015, 'TER': 12.672701286326824}


,bleu,rouge1,rouge2,TER
0,84.32542,0.95095,0.895144,7.657658
